# Prediction with LSTM model (v4.5.6)
<p style='text-align: right;'>with selectd5.csv</p>

* history
  * 2025/05/23 PM06:02 : 3번째 모델의 3번째 데이터를 사용
  * 2025/05/27
  * 2025/05/30 : v4.5.1 작성 시작
  * 2025/06/04 : v4.5.2 last update
  * 2025/06/04 : v4.5.3 작성
  * 2025/06/05 : v4.5.4 작성
  * 2025/06/05 : v4.5.5 작성 model_id=14
  * 2025/06/05 : v4.5.6 작성 model_id=4

In [8]:
!pip uninstall tensotflow

In [ ]:
!pip install scikit-learn
!pip install tensorflow-cpu
!pip install numpy pandas keras scikit-learn matplotlib scikeras
from datetime import datetime
print(f'restart kernel... {datetime.now()}')

In [13]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    from datetime import datetime
    print(f'restart kernel... {datetime.now()}')
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

restart_kernel()

import os
import sys
from datetime import datetime
cwd = os.getcwd()
sys.path.append(cwd)
print(sys.path)

restart kernel... 2025-06-06 14:58:01.525484
['/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/simpson/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/home/simpson/jupyter-workspace/LSTM_Analyze']


In [1]:
# load dependacies
import os
import time
import resource
from datetime import datetime
from lib.globalvar import *
from lib.data_loader import DataLoader
from lib.small_predict import predict_small, print_predict_small
from lib.activation import ActivationOutput, RecurrentActivation
from lib.small_predict import build_and_predict, gen_multi_model, save_model, generate_metric
from lib.common_env_sets import lstm_units, dense_units, steps, metrics, dropout
from lib.common_env_sets import learning_rate, last_lstm_return_sequences
from lib.common_env_sets import loss, output_dense_activation, epochs, rand_seed


print(f"imported library. ({datetime.now()})")

2025-06-06 14:58:16.788118: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-06 14:58:16.844726: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-06 14:58:16.845760: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-06 14:58:17.924400: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


imported library. (2025-06-06 14:58:19.371313)


In [2]:
##
## 모델링 환경 설정
##
window=1 # default = 1 , help = "time stamps"
data_dir='./csv/selectd5.csv'
    
#mode='predict' # help = "back-test or predict")
mode='back-test'
mode2='sampling' # help = "greed or sampling")
verb='verbose' # help = "verbose or not_verb")
    
training_length=0.95 # default = 0.9)
from_pos = 0
last = [[8, 11, 14, 17, 36, 39],
        [1, 5, 18, 20, 30, 35],
        [7, 9, 24, 40, 42, 44],
        [3, 6, 7, 11, 12, 17],
        [3, 13, 28, 34, 38, 42],
        [5, 12, 24, 29, 32, 42]]
print(f"finished to set environemnt. ({datetime.now()})")


finished to set environemnt. (2025-06-06 14:58:20.673589)


In [3]:
dataset_dicts = {
    1: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=0,
                  verbose=False
                  ),
    2: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=300,
                  verbose=False
                  )
    }

print(f'completed to load data. {datetime.now()}')

completed to load data. 2025-06-06 14:58:21.759462


In [4]:
###########
## layers
## LSTM Neural 계층 선언
###########
datasets = [dataset_dicts[1], # 0
            dataset_dicts[2], # 1
           ]

lstm_args = {4: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[1],
                          "steps": steps[0], # last output elements count
                          "metrics": [], #metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[2],
                          "dense_units": dense_units[2],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "return_state": False,
                          "rand_seed": rand_seed[0] # rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 0#90 #100
                           },
                 "dataset": datasets[0]
                 },
             }

print(f'completed to set env for all models. {datetime.now()}')

completed to set env for all models. 2025-06-06 14:58:22.504862


In [5]:
def gen_metric(test_id, gen_num, round_limit, lstm_args=lstm_args):
    """ gen_metric """
    model_version_from = 1
    model_version_end = 1
    print(f'gen_metric.step.0 [{datetime.now()}]')
    args = {'test_id': test_id,
            'version': model_version_from,
            'write_to_db': True,
            'last': last,
            'mode2': mode2,
            'gen_num': gen_num,
            'round_limit': round_limit,
            'db_file_name': "./db/metrics.db"
            }
    print(f'gen_metric.step.1 [{datetime.now()}]')
    generate_metric(args=args,
                    from_version=model_version_from,
                    to_version=model_version_end,
                    lstm_args=lstm_args)
    print(f'gen_metric.step.2 [{datetime.now()}]')

In [ ]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_1'
gen_num=5
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

In [8]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_2'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-05 16:57:44.558011]
gen_metric.step.1 [2025-06-05 16:57:44.558779]
Test Start: 1~1
CPU 시간: 200.544639 초
메모리 사용량: 912144 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 16:57:44.558881
********************
start to train all models. 2025-06-05 16:57:44.558898
4's training. status=start 2025-06-05 16:57:44.558913
4's training. status=end 2025-06-05 16:58:30.258066
completed to train all models. 2025-06-05 16:58:30.258388
train to create model(1). 2025-06-05 16:58:33.258477
CPU 시간: 397.729002 초
메모리 사용량: 1031424 KB
gen_metric.step.2 [2025-06-05 17:01:28.341745]


In [9]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_3'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-05 17:01:53.394111]
gen_metric.step.1 [2025-06-05 17:01:53.394460]
Test Start: 1~1
CPU 시간: 398.417202 초
메모리 사용량: 1031424 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 17:01:53.394550
********************
start to train all models. 2025-06-05 17:01:53.394567
4's training. status=start 2025-06-05 17:01:53.394582
4's training. status=end 2025-06-05 17:02:39.210146
completed to train all models. 2025-06-05 17:02:39.210259
train to create model(1). 2025-06-05 17:02:42.210350
CPU 시간: 594.614292 초
메모리 사용량: 1148016 KB
gen_metric.step.2 [2025-06-05 17:05:34.936140]


In [10]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_4'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-05 17:06:07.280569]
gen_metric.step.1 [2025-06-05 17:06:07.280661]
Test Start: 1~1
CPU 시간: 595.392947 초
메모리 사용량: 1148016 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 17:06:07.280748
********************
start to train all models. 2025-06-05 17:06:07.280764
4's training. status=start 2025-06-05 17:06:07.280779
4's training. status=end 2025-06-05 17:06:52.769013
completed to train all models. 2025-06-05 17:06:52.769115
train to create model(1). 2025-06-05 17:06:55.769206
CPU 시간: 793.187879 초
메모리 사용량: 1263536 KB
gen_metric.step.2 [2025-06-05 17:09:53.054340]


In [11]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_5'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-05 17:20:42.479797]
gen_metric.step.1 [2025-06-05 17:20:42.479888]
Test Start: 1~1
CPU 시간: 794.104507 초
메모리 사용량: 1263536 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 17:20:42.479975
********************
start to train all models. 2025-06-05 17:20:42.479993
4's training. status=start 2025-06-05 17:20:42.480008
4's training. status=end 2025-06-05 17:21:27.767116
completed to train all models. 2025-06-05 17:21:27.767218
train to create model(1). 2025-06-05 17:21:30.767306
CPU 시간: 991.305959 초
메모리 사용량: 1377712 KB
gen_metric.step.2 [2025-06-05 17:24:24.640008]


In [12]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_6'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-05 17:25:56.169507]
gen_metric.step.1 [2025-06-05 17:25:56.169730]
Test Start: 1~1
CPU 시간: 992.258392 초
메모리 사용량: 1377712 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 17:25:56.169792
********************
start to train all models. 2025-06-05 17:25:56.169804
4's training. status=start 2025-06-05 17:25:56.169813
4's training. status=end 2025-06-05 17:26:41.687751
completed to train all models. 2025-06-05 17:26:41.687853
train to create model(1). 2025-06-05 17:26:44.687932
CPU 시간: 1189.297003 초
메모리 사용량: 1493392 KB
gen_metric.step.2 [2025-06-05 17:29:41.196634]


In [13]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_7'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-05 17:29:44.240378]
gen_metric.step.1 [2025-06-05 17:29:44.240431]
Test Start: 1~1
CPU 시간: 1190.375658 초
메모리 사용량: 1493392 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 17:29:44.240489
********************
start to train all models. 2025-06-05 17:29:44.240500
4's training. status=start 2025-06-05 17:29:44.240509
4's training. status=end 2025-06-05 17:30:29.502494
completed to train all models. 2025-06-05 17:30:29.502596
train to create model(1). 2025-06-05 17:30:32.502684
CPU 시간: 1387.190363 초
메모리 사용량: 1608096 KB
gen_metric.step.2 [2025-06-05 17:33:26.997318]


In [14]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_8'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-05 17:33:29.129156]
gen_metric.step.1 [2025-06-05 17:33:29.129214]
Test Start: 1~1
CPU 시간: 1388.377748 초
메모리 사용량: 1608096 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 17:33:29.129272
********************
start to train all models. 2025-06-05 17:33:29.129283
4's training. status=start 2025-06-05 17:33:29.129292
4's training. status=end 2025-06-05 17:34:14.919217
completed to train all models. 2025-06-05 17:34:14.919318
train to create model(1). 2025-06-05 17:34:17.919399
CPU 시간: 1585.861168 초
메모리 사용량: 1721088 KB
gen_metric.step.2 [2025-06-05 17:37:13.477221]


In [7]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_9'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 10:29:14.458746]
gen_metric.step.1 [2025-06-06 10:29:14.458994]
Test Start: 1~1
CPU 시간: 1.328125 초
메모리 사용량: 371528 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 10:29:14.459051
********************
start to train all models. 2025-06-06 10:29:14.459060
4's training. status=start 2025-06-06 10:29:14.459067
4's training. status=end 2025-06-06 10:29:34.587033
completed to train all models. 2025-06-06 10:29:34.587115
train to create model(1). 2025-06-06 10:29:37.587305
INFO:tensorflow:Assets written to: ram://28ab2deca6ab4a709a58518443aacc1a/assets


INFO:tensorflow:Assets written to: ram://28ab2deca6ab4a709a58518443aacc1a/assets


CPU 시간: 46.296875 초
메모리 사용량: 1225712 KB
gen_metric.step.2 [2025-06-06 10:30:56.470593]


In [8]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_10'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 10:55:13.046049]
gen_metric.step.1 [2025-06-06 10:55:13.046439]
Test Start: 1~1
CPU 시간: 46.625 초
메모리 사용량: 1225712 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 10:55:13.046509
********************
start to train all models. 2025-06-06 10:55:13.046518
4's training. status=start 2025-06-06 10:55:13.046526
4's training. status=end 2025-06-06 10:55:34.321883
completed to train all models. 2025-06-06 10:55:34.321969
train to create model(1). 2025-06-06 10:55:37.322501
INFO:tensorflow:Assets written to: ram://566d74090e9d4956925b65b3e1a673c4/assets


INFO:tensorflow:Assets written to: ram://566d74090e9d4956925b65b3e1a673c4/assets


CPU 시간: 91.421875 초
메모리 사용량: 1244136 KB
gen_metric.step.2 [2025-06-06 10:56:57.462749]


In [9]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_11'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 11:05:14.122936]
gen_metric.step.1 [2025-06-06 11:05:14.123139]
Test Start: 1~1
CPU 시간: 91.8125 초
메모리 사용량: 1244136 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 11:05:14.123207
********************
start to train all models. 2025-06-06 11:05:14.123217
4's training. status=start 2025-06-06 11:05:14.123224
4's training. status=end 2025-06-06 11:05:35.304958
completed to train all models. 2025-06-06 11:05:35.305047
train to create model(1). 2025-06-06 11:05:38.305523
INFO:tensorflow:Assets written to: ram://690d073f51214d009ef0dbf25366e98f/assets


INFO:tensorflow:Assets written to: ram://690d073f51214d009ef0dbf25366e98f/assets


CPU 시간: 136.578125 초
메모리 사용량: 1268568 KB
gen_metric.step.2 [2025-06-06 11:06:58.821416]


In [11]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_12'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 11:08:56.851334]
gen_metric.step.1 [2025-06-06 11:08:56.851637]
Test Start: 1~1
CPU 시간: 137.03125 초
메모리 사용량: 1268568 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 11:08:56.851711
********************
start to train all models. 2025-06-06 11:08:56.851721
4's training. status=start 2025-06-06 11:08:56.851729
4's training. status=end 2025-06-06 11:09:17.765127
completed to train all models. 2025-06-06 11:09:17.765220
train to create model(1). 2025-06-06 11:09:20.765320
INFO:tensorflow:Assets written to: ram://5f2b9f08c334488da5b174ae25b64fab/assets


INFO:tensorflow:Assets written to: ram://5f2b9f08c334488da5b174ae25b64fab/assets


CPU 시간: 180.890625 초
메모리 사용량: 1285488 KB
gen_metric.step.2 [2025-06-06 11:10:44.364457]


In [10]:
# 데이터 셋을 수정해서 테스트 합니다.
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_14'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 11:53:57.390938]
gen_metric.step.1 [2025-06-06 11:53:57.391259]
Test Start: 1~1
CPU 시간: 36.859375 초
메모리 사용량: 953264 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 11:53:57.391346
********************
start to train all models. 2025-06-06 11:53:57.391361
4's training. status=start 2025-06-06 11:53:57.391369
4's training. status=end 2025-06-06 11:54:17.761072
completed to train all models. 2025-06-06 11:54:17.761150
train to create model(1). 2025-06-06 11:54:20.761244
INFO:tensorflow:Assets written to: ram://bed8a096ae634883bf374a890b1b453c/assets


INFO:tensorflow:Assets written to: ram://bed8a096ae634883bf374a890b1b453c/assets


CPU 시간: 82.296875 초
메모리 사용량: 1462756 KB
gen_metric.step.2 [2025-06-06 11:55:39.381164]


In [11]:
# 데이터 셋을 수정해서 테스트 합니다.
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_15'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 11:56:24.059200]
gen_metric.step.1 [2025-06-06 11:56:24.059614]
Test Start: 1~1
CPU 시간: 82.765625 초
메모리 사용량: 1462756 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 11:56:24.059675
********************
start to train all models. 2025-06-06 11:56:24.059683
4's training. status=start 2025-06-06 11:56:24.059691
4's training. status=end 2025-06-06 11:56:44.284243
completed to train all models. 2025-06-06 11:56:44.284348
train to create model(1). 2025-06-06 11:56:47.284416
INFO:tensorflow:Assets written to: ram://2ba16ac61c6e4ef190e64a1e448762d3/assets


INFO:tensorflow:Assets written to: ram://2ba16ac61c6e4ef190e64a1e448762d3/assets


CPU 시간: 129.296875 초
메모리 사용량: 1477984 KB
gen_metric.step.2 [2025-06-06 11:58:06.388850]


In [12]:
# 데이터 셋을 수정해서 테스트 합니다.
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_16'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 12:00:16.929338]
gen_metric.step.1 [2025-06-06 12:00:16.929524]
Test Start: 1~1
CPU 시간: 129.859375 초
메모리 사용량: 1477984 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 12:00:16.929581
********************
start to train all models. 2025-06-06 12:00:16.929589
4's training. status=start 2025-06-06 12:00:16.929596
4's training. status=end 2025-06-06 12:00:37.109211
completed to train all models. 2025-06-06 12:00:37.109289
train to create model(1). 2025-06-06 12:00:40.109427
INFO:tensorflow:Assets written to: ram://830763330d4d4bf2947f9860bc638fe6/assets


INFO:tensorflow:Assets written to: ram://830763330d4d4bf2947f9860bc638fe6/assets


CPU 시간: 177.375 초
메모리 사용량: 1501152 KB
gen_metric.step.2 [2025-06-06 12:02:00.606857]


In [15]:
# 데이터 셋을 수정해서 테스트 합니다.
# predict -> back_test
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_17'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 12:06:53.196224]
gen_metric.step.1 [2025-06-06 12:06:53.196555]
Test Start: 1~1
CPU 시간: 177.890625 초
메모리 사용량: 1501152 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 12:06:53.196626
********************
start to train all models. 2025-06-06 12:06:53.196635
4's training. status=start 2025-06-06 12:06:53.196642
4's training. status=end 2025-06-06 12:07:13.545985
completed to train all models. 2025-06-06 12:07:13.546063
train to create model(1). 2025-06-06 12:07:16.546117
INFO:tensorflow:Assets written to: ram://a419ce4ebd624f75b73105c7acc93e25/assets


INFO:tensorflow:Assets written to: ram://a419ce4ebd624f75b73105c7acc93e25/assets


CPU 시간: 225.328125 초
메모리 사용량: 1512596 KB
gen_metric.step.2 [2025-06-06 12:08:36.618191]


In [16]:
# 데이터 셋을 수정해서 테스트 합니다.
# predict -> back_test
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_18'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 12:09:30.408520]
gen_metric.step.1 [2025-06-06 12:09:30.409056]
Test Start: 1~1
CPU 시간: 225.84375 초
메모리 사용량: 1512596 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 12:09:30.409136
********************
start to train all models. 2025-06-06 12:09:30.409145
4's training. status=start 2025-06-06 12:09:30.409153
4's training. status=end 2025-06-06 12:09:50.841610
completed to train all models. 2025-06-06 12:09:50.841697
train to create model(1). 2025-06-06 12:09:53.841756
INFO:tensorflow:Assets written to: ram://812eef8f1ef7477d827048e21266e3aa/assets


INFO:tensorflow:Assets written to: ram://812eef8f1ef7477d827048e21266e3aa/assets


CPU 시간: 271.734375 초
메모리 사용량: 1531904 KB
gen_metric.step.2 [2025-06-06 12:11:13.359821]


In [17]:
# 데이터 셋을 수정해서 테스트 합니다.
# predict -> back_test
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_19'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 12:11:52.014251]
gen_metric.step.1 [2025-06-06 12:11:52.014629]
Test Start: 1~1
CPU 시간: 272.21875 초
메모리 사용량: 1531904 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 12:11:52.014725
********************
start to train all models. 2025-06-06 12:11:52.014739
4's training. status=start 2025-06-06 12:11:52.014750
4's training. status=end 2025-06-06 12:12:14.147610
completed to train all models. 2025-06-06 12:12:14.147698
train to create model(1). 2025-06-06 12:12:17.148047
INFO:tensorflow:Assets written to: ram://97f6ad5f809942c0a2808383521f0de7/assets


INFO:tensorflow:Assets written to: ram://97f6ad5f809942c0a2808383521f0de7/assets


CPU 시간: 316.28125 초
메모리 사용량: 1542244 KB
gen_metric.step.2 [2025-06-06 12:13:37.275269]


In [18]:
# 데이터 셋을 수정해서 테스트 합니다.
# predict -> back_test
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_20'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 12:19:22.204116]
gen_metric.step.1 [2025-06-06 12:19:22.204307]
Test Start: 1~1
CPU 시간: 316.78125 초
메모리 사용량: 1542244 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 12:19:22.204392
********************
start to train all models. 2025-06-06 12:19:22.204402
4's training. status=start 2025-06-06 12:19:22.204410
4's training. status=end 2025-06-06 12:19:43.673801
completed to train all models. 2025-06-06 12:19:43.673881
train to create model(1). 2025-06-06 12:19:46.673990
INFO:tensorflow:Assets written to: ram://4126f40db62649ceb86b2d66372e74bd/assets


INFO:tensorflow:Assets written to: ram://4126f40db62649ceb86b2d66372e74bd/assets


CPU 시간: 361.921875 초
메모리 사용량: 1565580 KB
gen_metric.step.2 [2025-06-06 12:21:09.064118]


In [19]:
# 데이터 셋을 수정해서 테스트 합니다.
# predict -> back_test
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_21'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 12:22:46.984245]
gen_metric.step.1 [2025-06-06 12:22:46.984668]
Test Start: 1~1
CPU 시간: 362.40625 초
메모리 사용량: 1565580 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 12:22:46.984746
********************
start to train all models. 2025-06-06 12:22:46.984762
4's training. status=start 2025-06-06 12:22:46.984772
4's training. status=end 2025-06-06 12:23:08.736019
completed to train all models. 2025-06-06 12:23:08.736114
train to create model(1). 2025-06-06 12:23:11.736550
INFO:tensorflow:Assets written to: ram://076e5b08aa17449fa552063d947d1b23/assets


INFO:tensorflow:Assets written to: ram://076e5b08aa17449fa552063d947d1b23/assets


CPU 시간: 407.578125 초
메모리 사용량: 1577572 KB
gen_metric.step.2 [2025-06-06 12:24:35.989914]


In [20]:
# 데이터 셋을 수정해서 테스트 합니다.
# predict -> back_test
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_22'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 12:25:05.481278]
gen_metric.step.1 [2025-06-06 12:25:05.481553]
Test Start: 1~1
CPU 시간: 408.046875 초
메모리 사용량: 1577572 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 12:25:05.481614
********************
start to train all models. 2025-06-06 12:25:05.481624
4's training. status=start 2025-06-06 12:25:05.481633
4's training. status=end 2025-06-06 12:25:27.039859
completed to train all models. 2025-06-06 12:25:27.039971
train to create model(1). 2025-06-06 12:25:30.040164
INFO:tensorflow:Assets written to: ram://a3e12ba900de4b97a68bbb8266e04893/assets


INFO:tensorflow:Assets written to: ram://a3e12ba900de4b97a68bbb8266e04893/assets


CPU 시간: 453.640625 초
메모리 사용량: 1594868 KB
gen_metric.step.2 [2025-06-06 12:26:52.584764]


In [7]:
# 데이터 셋을 수정해서 테스트 합니다.
# predict -> back_test
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_23'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 13:55:30.372343]
gen_metric.step.1 [2025-06-06 13:55:30.372521]
Test Start: 1~1
CPU 시간: 1.15625 초
메모리 사용량: 394800 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 13:55:30.372576
********************
start to train all models. 2025-06-06 13:55:30.372585
4's training. status=start 2025-06-06 13:55:30.372592


2025-06-06 13:55:30.425428: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


4's training. status=end 2025-06-06 13:55:39.249418
completed to train all models. 2025-06-06 13:55:39.249496
train to create model(1). 2025-06-06 13:55:42.249562
INFO:tensorflow:Assets written to: ram://771a1aefd48643edb7acdd295c0cee8b/assets


INFO:tensorflow:Assets written to: ram://771a1aefd48643edb7acdd295c0cee8b/assets


CPU 시간: 38.015625 초
메모리 사용량: 1235888 KB
gen_metric.step.2 [2025-06-06 13:56:34.025344]


In [8]:
# 데이터 셋을 수정해서 테스트 합니다.
# predict -> back_test
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_24'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 13:56:45.520247]
gen_metric.step.1 [2025-06-06 13:56:45.520524]
Test Start: 1~1
CPU 시간: 38.390625 초
메모리 사용량: 1235888 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 13:56:45.520588
********************
start to train all models. 2025-06-06 13:56:45.520597
4's training. status=start 2025-06-06 13:56:45.520605
4's training. status=end 2025-06-06 13:56:54.536552
completed to train all models. 2025-06-06 13:56:54.536627
train to create model(1). 2025-06-06 13:56:57.536713
INFO:tensorflow:Assets written to: ram://e7b677a7633f4912b4b9d8301d719165/assets


INFO:tensorflow:Assets written to: ram://e7b677a7633f4912b4b9d8301d719165/assets


CPU 시간: 75.671875 초
메모리 사용량: 1241748 KB
gen_metric.step.2 [2025-06-06 13:57:49.564041]


In [9]:
# 데이터 셋을 수정해서 테스트 합니다.
# predict -> back_test
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_25'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 14:05:58.009051]
gen_metric.step.1 [2025-06-06 14:05:58.009376]
Test Start: 1~1
CPU 시간: 76.046875 초
메모리 사용량: 1241748 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 14:05:58.009459
********************
start to train all models. 2025-06-06 14:05:58.009468
4's training. status=start 2025-06-06 14:05:58.009476
4's training. status=end 2025-06-06 14:06:07.219001
completed to train all models. 2025-06-06 14:06:07.219078
train to create model(1). 2025-06-06 14:06:10.219236
INFO:tensorflow:Assets written to: ram://7c9a60f95c054c86a75446ecb628cd25/assets


INFO:tensorflow:Assets written to: ram://7c9a60f95c054c86a75446ecb628cd25/assets


CPU 시간: 112.9375 초
메모리 사용량: 1248120 KB
gen_metric.step.2 [2025-06-06 14:07:02.132138]


In [10]:
# 데이터 셋을 수정해서 테스트 합니다.
# predict -> back_test
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_26'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 14:07:04.621420]
gen_metric.step.1 [2025-06-06 14:07:04.621492]
Test Start: 1~1
CPU 시간: 113.296875 초
메모리 사용량: 1248120 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 14:07:04.621554
********************
start to train all models. 2025-06-06 14:07:04.621563
4's training. status=start 2025-06-06 14:07:04.621571
4's training. status=end 2025-06-06 14:07:13.974670
completed to train all models. 2025-06-06 14:07:13.974755
train to create model(1). 2025-06-06 14:07:16.974827
INFO:tensorflow:Assets written to: ram://d41d6c0103334227b2f06950c4aa1f95/assets


INFO:tensorflow:Assets written to: ram://d41d6c0103334227b2f06950c4aa1f95/assets


CPU 시간: 149.578125 초
메모리 사용량: 1253124 KB
gen_metric.step.2 [2025-06-06 14:08:09.478106]


In [11]:
# 데이터 셋을 수정해서 테스트 합니다.
# predict -> back_test
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_27'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 14:10:09.510010]
gen_metric.step.1 [2025-06-06 14:10:09.510344]
Test Start: 1~1
CPU 시간: 149.9375 초
메모리 사용량: 1253124 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 14:10:09.510411
********************
start to train all models. 2025-06-06 14:10:09.510420
4's training. status=start 2025-06-06 14:10:09.510428
4's training. status=end 2025-06-06 14:10:19.202376
completed to train all models. 2025-06-06 14:10:19.203291
train to create model(1). 2025-06-06 14:10:22.203689
INFO:tensorflow:Assets written to: ram://75ead2907f6e49d2bbe102090d5605b5/assets


INFO:tensorflow:Assets written to: ram://75ead2907f6e49d2bbe102090d5605b5/assets


CPU 시간: 185.5625 초
메모리 사용량: 1256160 KB
gen_metric.step.2 [2025-06-06 14:11:15.668194]


In [12]:
# 데이터 셋을 수정해서 테스트 합니다.
# predict -> back_test
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_6_28'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 14:11:42.737554]
gen_metric.step.1 [2025-06-06 14:11:42.737950]
Test Start: 1~1
CPU 시간: 185.921875 초
메모리 사용량: 1256160 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 14:11:42.738016
********************
start to train all models. 2025-06-06 14:11:42.738026
4's training. status=start 2025-06-06 14:11:42.738033
4's training. status=end 2025-06-06 14:11:52.187389
completed to train all models. 2025-06-06 14:11:52.187500
train to create model(1). 2025-06-06 14:11:55.187649
INFO:tensorflow:Assets written to: ram://e9b8c9f30d4e4b7eb74c7424b33d7510/assets


INFO:tensorflow:Assets written to: ram://e9b8c9f30d4e4b7eb74c7424b33d7510/assets


CPU 시간: 224.015625 초
메모리 사용량: 1266448 KB
gen_metric.step.2 [2025-06-06 14:12:47.605228]


In [ ]:
## end